In [1]:
!pip install bayesian-optimization

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn  as sns

from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix
from bayes_opt import BayesianOptimization
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
df= pd.read_csv("/content/drive/MyDrive/ML/heart.csv")

In [7]:
df

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [8]:
df.isnull().sum()

age         0
sex         0
cp          0
trtbps      0
chol        0
fbs         0
restecg     0
thalachh    0
exng        0
oldpeak     0
slp         0
caa         0
thall       0
output      0
dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trtbps    303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalachh  303 non-null    int64  
 8   exng      303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slp       303 non-null    int64  
 11  caa       303 non-null    int64  
 12  thall     303 non-null    int64  
 13  output    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [10]:
df1 = df.sample(frac=1).reset_index(drop=True)

In [11]:
df1.head(10)

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,58,1,2,140,211,1,0,165,0,0.0,2,0,2,1
1,43,1,0,110,211,0,1,161,0,0.0,2,0,3,1
2,52,1,0,128,204,1,1,156,1,1.0,1,0,0,0
3,60,0,2,102,318,0,1,160,0,0.0,2,1,2,1
4,52,1,1,128,205,1,1,184,0,0.0,2,0,2,1
5,67,0,2,152,277,0,1,172,0,0.0,2,1,2,1
6,44,1,1,120,263,0,1,173,0,0.0,2,0,3,1
7,61,1,0,120,260,0,1,140,1,3.6,1,1,3,0
8,62,0,0,124,209,0,1,163,0,0.0,2,0,2,1
9,48,1,1,110,229,0,1,168,0,1.0,0,0,3,0


In [12]:
X= df1.drop(columns=['output'])
y= df1['output']

In [13]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [14]:
scaler1= MinMaxScaler()
MinMax_scaled_X= scaler1.fit_transform(X)
MinMax_scaled_X

array([[0.60416667, 1.        , 0.66666667, ..., 1.        , 0.        ,
        0.66666667],
       [0.29166667, 1.        , 0.        , ..., 1.        , 0.        ,
        1.        ],
       [0.47916667, 1.        , 0.        , ..., 0.5       , 0.        ,
        0.        ],
       ...,
       [0.58333333, 1.        , 0.        , ..., 0.5       , 0.25      ,
        1.        ],
       [0.52083333, 0.        , 0.66666667, ..., 1.        , 0.        ,
        0.66666667],
       [0.83333333, 0.        , 1.        , ..., 1.        , 0.5       ,
        0.66666667]])

<font color= "lime"> <h2> Random Search

In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(MinMax_scaled_X,y,test_size= 20,random_state=0)

In [ ]:
rf_param_grid = {
                 'max_depth' : list(range(8,100,10)),
                 'n_estimators': list(range(50,2000,100)),
                 'min_samples_split': [2,6,8],
                 'min_samples_leaf': [2,6,8],
                 'bootstrap': [True, False]
                 }

In [ ]:
rf_param_grid

In [ ]:
m = RandomForestClassifier(n_jobs= 4)   # use -1 for full CPU utilization

In [ ]:
# m_r = GridSearchCV(param_grid=rf_param_grid, estimator = m, scoring = "accuracy", cv = 4)

In [ ]:
m_r = RandomizedSearchCV(param_distributions=rf_param_grid, estimator = m, scoring = "accuracy", cv = 4,n_jobs=-1,n_iter=10)

In [ ]:
%time m_r.fit(X_train,y_train)

In [ ]:
m_r.best_params_

In [ ]:
for param, score in zip(m_r.cv_results_['params'], m_r.cv_results_['mean_test_score']):
    print(param, score)

In [ ]:
rf_bp = m_r.best_params_

In [ ]:
rf_classifier=RandomForestClassifier(n_estimators=rf_bp["n_estimators"],
                                     min_samples_split=rf_bp['min_samples_split'],
                                     min_samples_leaf=rf_bp['min_samples_leaf'],
                                     max_depth=rf_bp['max_depth'],
                                     bootstrap=rf_bp['bootstrap'])

In [ ]:
rf_classifier.fit(X_train,y_train)

In [ ]:
y_pred_randomCV = rf_classifier.predict(X_test)

In [ ]:
y_pred_randomCV

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

y_pred_randomCV = rf_classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred_randomCV))
print(accuracy_score(y_test,y_pred_randomCV))
print(classification_report(y_test,y_pred_randomCV))
acc = accuracy_score(y_test,y_pred_randomCV)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score



print("accuracy: ",accuracy_score(y_test,y_pred_randomCV))
print("precision_score: ",precision_score(y_test,y_pred_randomCV))
print("recall_score: ",recall_score(y_test,y_pred_randomCV))
print("f1_score: ",f1_score(y_test,y_pred_randomCV))
print("roc_auc_score: ",roc_auc_score(y_test,y_pred_randomCV))

<font color= "lime"> <h2> Baysian Optimization

Bayesian Optimization uses probability to find the minimum of a function. The final airm is to find the input value to a function which can give us the lowest possible output value. There are 3 main paramaters in Bayesian Optimization.
1. Objective function: loss function to minimize
2. Domain Space: range of input values
3. Optimization algorithm: search algorithm
Bayesian Optimization builds a probability model of the objective function and uses it to select hyperparameter to evaluate in the true objective function


description:

hp --> for defining hyperparameter values
STATUS_OK --> evaluation successfully done or not
Trials --> used for minimizing objective function
fmin --> minimize function
TPE (Tree-structured Parzen Estimator) --> Search algorithm

In [15]:
!pip install hyperopt

In [16]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

In [17]:
space = {'criterion': hp.choice('criterion', ['gini', 'entropy']),
        'n_estimators': hp.choice('n_estimators',[ 10,50,100,200]),
        'min_samples_leaf': hp.choice('min_samples_leaf',[ 1,2,3]),
        'min_samples_split' : hp.choice ('min_samples_split',[ 2,3,4]),
        'max_depth' : hp.choice ('max_depth', [3,5,10]),
        'max_features': hp.choice('max_features', ['sqrt', 'log2', None]),
        'max_leaf_nodes' : hp.choice ('max_leaf_nodes',[2,3,4, None])
    }

In [18]:
space

{'criterion': <hyperopt.pyll.base.Apply at 0x7a08fb3ad510>,
 'n_estimators': <hyperopt.pyll.base.Apply at 0x7a08fb3ad6f0>,
 'min_samples_leaf': <hyperopt.pyll.base.Apply at 0x7a08fb3ad120>,
 'min_samples_split': <hyperopt.pyll.base.Apply at 0x7a08fb3ae7d0>,
 'max_depth': <hyperopt.pyll.base.Apply at 0x7a08fb3acc10>,
 'max_features': <hyperopt.pyll.base.Apply at 0x7a08fb3acd90>,
 'max_leaf_nodes': <hyperopt.pyll.base.Apply at 0x7a08fb3ad060>}

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score


def objective(space):

  RF_classifier = RandomForestClassifier(criterion= space['criterion'],
                                           n_estimators= space['n_estimators'],
                                           min_samples_split= space['min_samples_split'],
                                           min_samples_leaf= space['min_samples_leaf'],
                                           max_depth= space['max_depth'],
                                           max_features= space['max_features'],
                                           max_leaf_nodes= space['max_leaf_nodes'])

  accuracy = cross_val_score(RF_classifier, X_train, y_train, cv = 5).mean()

  # We aim to maximize accuracy, therefore we return it as a negative value
  return {'loss': -accuracy, 'status': STATUS_OK }

In [22]:
trials = Trials()
best = fmin(fn= objective,      # fmin minimize the objective function fn
            space= space,
            algo= tpe.suggest,    # tpe.suggest refers to the process of suggesting the next set of hyperparameters using tpe algo
            max_evals = 80,
            trials= trials)
best

100%|██████████| 80/80 [01:34<00:00,  1.19s/trial, best loss: -0.8484335839598998]


{'criterion': 0,
 'max_depth': 1,
 'max_features': 0,
 'max_leaf_nodes': 3,
 'min_samples_leaf': 1,
 'min_samples_split': 1,
 'n_estimators': 3}

In [23]:
# optcome is index sometimes. So we need to find the index value by using mapping (inverse key value)

cri = {0:'gini', 1:'entropy'}
est= {0:10, 1:50, 2:100, 3:200}
min_split = {0:2, 1:3, 2:4}
min_leaf= {0:1, 1:2, 2:3}
depth= {0:3, 1:5, 2:10}
fea = {0:'sqrt', 1:'log2', 2:None}
max_leaf= {0:2, 1:3, 2:4, 3: None}


In [25]:
trainedRF = RandomForestClassifier(   criterion= cri[best['criterion']],                          # best['criterion'] = 1     cri[1]= "entropy"
                                      n_estimators= est[best['n_estimators']],                    # best['n_estimators']= 3   est[3]= 200
                                      min_samples_split= min_split[best['min_samples_split']],
                                      min_samples_leaf= best['min_samples_leaf'],
                                      max_depth= depth[best['max_depth']],                # best['max_depth'] =1      depth[1] =5
                                      max_features= fea[best['max_features']],
                                      max_leaf_nodes= best['max_leaf_nodes']).fit(X_train,y_train)

In [26]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

predictionGB = trainedRF.predict(X_test)
print(confusion_matrix(y_test,predictionGB))
print(accuracy_score(y_test,predictionGB))
print(classification_report(y_test,predictionGB))
acc = accuracy_score(y_test,predictionGB)

[[5 6]
 [0 9]]
0.7
              precision    recall  f1-score   support

           0       1.00      0.45      0.62        11
           1       0.60      1.00      0.75         9

    accuracy                           0.70        20
   macro avg       0.80      0.73      0.69        20
weighted avg       0.82      0.70      0.68        20



In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score



print("accuracy: ",accuracy_score(y_test,predictionGB))
print("precision_score: ",precision_score(y_test,predictionGB))
print("recall_score: ",recall_score(y_test,predictionGB))
print("f1_score: ",f1_score(y_test,predictionGB))
print("roc_auc_score: ",roc_auc_score(y_test,predictionGB))

accuracy:  0.7
precision_score:  0.6
recall_score:  1.0
f1_score:  0.7499999999999999
roc_auc_score:  0.7272727272727273
